# 生成学习算法

以线性回归、逻辑回归为代表的广义线性模型，都是在针对 $p(y|x;\theta)$，也就是给定 $x$ 时 $y$ 的条件概率分布进行建模。

以分类问题为例，给定一个训练集，逻辑回归，试图寻找一条直线——也就是决策边界——来区分正类和反类。模型建立后，对于一个新的测试样本，给定其特征，我们判定这个样本落在决策边界的哪一侧，然后据此做出预测。

直接学习 $p(y|x)$ 的算法，或者说直接将输入空间 $\chi$ 映射到标签 $\{0, 1\}$的算法，称为**判别学习算法 Discriminative Learning Algorithm**。与此相反，针对 $p(x|y)$ 建模的算法，称为**生成学习算法 Generative Learning Algorithm**。

还是以分类问题为例，生成学习算法通过对训练集的学习，给定一个分类 $y \in \{0, 1\}$，我们的模型试图描述这个分类对应的特征的分布。

通过对 $p(y)$ （称为**先验概率 class prior**）和 $p(x|y)$ 建模，我们之后通过贝叶斯法则来计算给定 $x$ 时 $y$ 的后验分布：
$$ p(y|x) = \frac{p(x|y)p(y)}{p(x)} $$

其中，分母 $p(x) = p(x|y=1)p(y=1)+p(x|y=0)p(y=0)$。而实际上，在预测时，我们并不需要计算 $p(x)$，因为
$$
\begin{split}
arg \max_yp(y|x) &= arg \max_y\frac{p(x|y)p(y)}{p(x)} \\
&= arg \max_yp(x|y)p(y)
\end{split}
$$

本节包括以下内容：
1. 高斯判别分析 Gaussian discriminant analysis
2. 朴素贝叶斯 Naive Bayes

### 1. 高斯判别分析 Gaussian Discriminant Analysis

在高斯判别分析中，我们假设 $p(x|y)$ 服从多维正态分布。

#### 1.1 多维正态分布 The MultiVariate Normal Distribution

多维正态分布也称作多维高斯分布，其参数包括**均值向量 mean vector** $\mu \in \mathbb{R}^n$ 以及**协方差矩阵 covariance matrix** $\Sigma \in \mathbb{R}^{n \times n}$。

关于多维正态分布的具体特征，可参见[随机变量及其数字特征-矩、协方差矩阵](http://nbviewer.jupyter.org/github/reata/ProbabilityAndStatistics/blob/master/Mathematical%20Properties%20of%20Random%20Variable.ipynb)，这里略过详细介绍。


#### 1.2 高斯判别分析模型 The Gaussian Discriminant Analysis Model

对于特征 $x$ 为连续型变量的分类问题，我们可以使用高斯判别分析（GDA）模型。该模型的假设包括：
$$
\begin{split}
y &\sim Bernoulli(\phi) \\
x|y=0 &\sim N(\mu_0, \Sigma) \\
x|y=1 &\sim N(\mu_1, \Sigma) \\
\end{split}
$$
对应的概率质量函数/概率密度函数为
$$
\begin{split}
p(y) &= \phi^y(1-\phi)^{1-y} \\
p(x|y=0) &= \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}exp(-\frac{1}{2}(x-\mu_0)^{T}\Sigma^{-1}(x-\mu_0)) \\
p(x|y=1) &= \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}exp(-\frac{1}{2}(x-\mu_1)^{T}\Sigma^{-1}(x-\mu_1)) \\
\end{split}
$$

这里模型的参数为 $\phi, \Sigma, \mu_0, \mu_1$。（注意到尽管我们使用了不同的均值向量 $\mu_0$ 和 $\mu_1$，这里通常只使用同一个协方差矩阵 $\Sigma$）其对数似然函数为
$$
\begin{split}
\ell(\phi, \mu_0, \mu_1, \Sigma) &= log\prod_{i=1}^m p(x^{(i)}, y^{(i)}; \phi, \mu_0, \mu_1, \Sigma) \\
&= log\prod_{i=1}^m p(x^{(i)}|y^{(i)}; \mu_0, \mu_1, \Sigma)p(y^{(i)};\phi)
\end{split}
$$

使 $\ell$ 最大的各参数分别为
$$
\begin{split}
\phi &= \frac{1}{m}\sum_{i=1}^m 1\{y^{(i)}=1\} \\
\mu_0 &= \frac{\sum_{i=1}^m 1\{y^{(i)}=0\}x^{(i)}}{\sum_{i=1}^m 1\{y^{(i)}=0\}} \\
\mu_1 &= \frac{\sum_{i=1}^m 1\{y^{(i)}=1\}x^{(i)}}{\sum_{i=1}^m 1\{y^{(i)}=1\}} \\
\Sigma &= \frac{1}{m} \sum_{i=1}^m (x^{(i)}-\mu_{y^{(i)}})(x^{(i)}-\mu_{y^{(i)}})^T
\end{split}
$$

由于使用了相同的协方差，最终根据 $y=1$ 和 $y=0$ 估计出的多维高斯分布，形状将完全相同，区别仅仅在于均值。而两个高斯分布的等差线相交的地方，就是高斯判别分析模型的决策边界。

#### 1.3 高斯判别分析和逻辑回归 GDA and Logistic Regression

高斯判别分析和逻辑回归之间，存在着非常有意思的关联。如果我们将 $p(y=1|x;\phi, \mu_0, \mu_1, \Sigma)$ 视为 $x$ 的函数，即
$$ p(y=1|x) = \frac{p(x|y=1)p(y=1)}{p(x|y=1) + p(x|y=0)} $$

则这个函数可以写成
$$ p(y=1|x;\phi, \mu_0, \mu_1, \Sigma) = \frac{1}{1+exp(-\theta^Tx)} $$
的形式，其中，$\theta$ 是 $\phi, \Sigma, \mu_0, \mu_1$ 的函数。这正是作为一种判别学习算法的逻辑回归对 $p(y=1|x)$ 的建模形式。

但是，高斯判别分析和逻辑回归求出的决策边界通常是不同的，也就是说 $\theta$ 的值是不同的，该如何区分两种模型的优劣呢？

注意我们上面的结论是，当 $p(x|y)$ 服从多维高斯分布时，则 $p(y|x)$ 必然是sigmoid函数的形式。而这个命题的逆命题是错误的，也就是说，当 $p(y|x)$ 是sigmoid函数的形式时，并不能得出 $p(x|y)$ 服从多维高斯分布。换言之，高斯判别分析的假设，比逻辑回归更强。

当高斯判别分析的假设成立时，它会找到比逻辑回归更好的模型，高斯判别分析是一个**渐进有效估计量 asymptotically efficient estimator**，渐进有效估计量的含义是，在大样本的条件下，没有其它算法可以严格意义上比高斯判别分析更准确地预测 $p(y|x)$。但同时，当假设成立，即使数据量小，高斯判别分析的效果通常也好于逻辑回归。

相反，由于使用了更弱的假设约束，逻辑回归会更健壮，不容易受到错误假设的影响。有很多种不同的假设（其中包括所有的指数分布族），都会使得 $p(y|x)$ 呈现sigmoid函数的形态。譬如一个数据集的 $p(x|y)$ 服从泊松分布，其 $p(y|x)$ 为sigmoid函数，当我们使用高斯判别分析来建模时，最终结果会很难预料，但我们可以预计逻辑回归仍然可以正常预测。

总结来说，当数据并不是高斯分布时，数据量较大的情况下，逻辑回归几乎总是比高斯判别分析表现得更好。出于这个原因，在实际项目中逻辑回归的使用远远多于高斯判别分析。这个区别，或多或少也是判别算法和生成算法的区别，机关如此，我们接下来介绍的朴素贝叶斯算法，依然是非常经典且流行的分类算法。